## lightGBM(Reg) + tf-idf(max) + sudachi(C)

https://blog.amedama.jp/entry/lightgbm-cv-model

In [ ]:
# setup lib
! pip install scikit-learn optuna lightgbm pandas
# optuna/lightgbm(gpu)
#! pip uninstall --yes lightgbm && pip install --install-option=--gpu lightgbm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 348 kB 8.0 MB/s 
     |████████████████████████████████| 81 kB 9.2 MB/s 
     |████████████████████████████████| 41.6 MB 1.3 MB/s 
     |████████████████████████████████| 210 kB 54.1 MB/s 
     |████████████████████████████████| 78 kB 7.9 MB/s 
     |████████████████████████████████| 50 kB 6.4 MB/s 
     |████████████████████████████████| 147 kB 40.9 MB/s 
     |████████████████████████████████| 112 kB 60.5 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=19c169e30cda310d7911a21168ec9cc471b0d70c99a5b26e1bf4c4321cb8a4cc
  Stored in directory: /root/.cache/pip/wheels/7f/1a/65/84ff8c386bec21fca6d220ea1f5498a0367883a78dd5ba6122
Successfully built pyperclip
  Attempting uninstall: scipy
    Found existing installation: scipy 1.3.0
    Uninstalling scipy-1.3.0:
      Successfully uninstalled scipy-1.3.0
  At

In [ ]:
# ライブラリの導入

import numbers
import numpy as np
import pandas as pd

import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import cohen_kappa_score
from functools import partial
import scipy as sp
def adapt_labels(labels):
  max_label = np.max(labels) # 2 or 4

  if max_label == 2:
    labels += 2
  elif max_label == 4:
    labels -= 2
  else:
    print('Exception error! please check with np.unique(labels).')
  return labels

In [ ]:
#ファイル
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/grad_comp

Mounted at /content/drive
/content/drive/MyDrive/grad_comp


## load data

In [ ]:
# preprocess path
prepro = 'sudachi/C'
# n_features = 10000

In [ ]:
# data

f = open('preprocess/' + prepro + '/text.prep_train.txt', 'r')
train_data = f.read()
train_data = train_data.split('\n')
del train_data[30000]
#x_train = np.zeros((30000, len(res)))

f = open('preprocess/' + prepro + '/text.prep_test.txt', 'r')
test_data = f.read()
test_data = test_data.split('\n')
del test_data[2500]
#x_test = np.zeros((2500, len(res)))

f = open('preprocess/' + prepro + '/text.prep_dev.txt', 'r')
dev_data = f.read()
dev_data = dev_data.split('\n')
del dev_data[2500]
#x_dev = np.zeros((2500, len(res)))

# label
y_train = np.loadtxt('/content/drive/MyDrive/grad_comp/data/label.train.txt')
y_dev = np.loadtxt('/content/drive/MyDrive/grad_comp/data/label.dev.txt')
adapt_labels(y_train)
adapt_labels(y_dev)

u, counts = np.unique(y_train, return_counts=True)
print(u)
print(counts)

[0. 1. 2. 3. 4.]
[3543 5593 9227 7760 3877]


## TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
x_train = vectorizer.fit_transform(train_data)
x_dev = vectorizer.transform(dev_data)
x_test = vectorizer.transform(test_data)

In [ ]:
x_train.shape

(30000, 26803)

QWK = 0.041  C = 0.01
QWK = 0.210  C = 0.1
QWK = 0.334  C = 1
QWK = 0.324  C = 10
QWK = 0.290  C = 100
最適なハイパーパラメタは C = 1

## lightGBM

In [ ]:
# 14 min
! pip uninstall --yes scipy
! pip install scipy==1.3.0
import scipy as sp

Found existing installation: scipy 1.7.3
Uninstalling scipy-1.7.3:
  Successfully uninstalled scipy-1.7.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 23.6 MB 1.4 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for scipy: filename=scipy-1.3.0-cp38-cp38-linux_x86_64.whl size=43877239 sha256=3960e488b346924c554e9dd5c5a8a55db45a4c420d71ba8e680fc3d1fb65bb08
  Stored in directory: /root/.cache/pip/wheels/2c/c1/a0/d7b4dc8a7b3c8e3cfa99b2098af0c94aadce5f5b516015d952
Successfully built scipy
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xarray-einstats 0.4.0 requires scipy>=1.6, but you have scipy 1.3.0 which is incompatible.
pymc 4.1.4 requires scipy>=1.4.1, but you have scipy

You must restart the runtime in order to use newly installed versions.

In [ ]:
sp.version.full_version

'1.3.0'

In [83]:
class OptimizedRounder(object):
    def __init__(self):
        self.coef_ = 0

    def _kappa_loss(self, coef, X, y):
        X_p = np.copy(X) # Xにも影響しないよう
        for i, pred in enumerate(X_p):
          # こちらはエラーなし
            if pred < coef[0]:
                X_p[i] = 0
            elif pred >= coef[0] and pred < coef[1]:
                X_p[i] = 1
            elif pred >= coef[1] and pred < coef[2]:
                X_p[i] = 2
            elif pred >= coef[2] and pred < coef[3]:
                X_p[i] = 3
            else:
                X_p[i] = 4
        # ライブラリののqwkに変更
        ll = cohen_kappa_score(y,X_p,weights='quadratic')
        return -ll # optimize.minimizeより

    # 閾値の最適化(nelder-mead法)
    def fit(self, X, y):
        loss_partial = partial(self._kappa_loss, X=X, y=y)
        initial_coef = [0.5, 1.5, 2.5, 3.5]
        x_qwk = sp.optimize.minimize(loss_partial, initial_coef,method='Nelder-Mead')
        #print(type(x_qwk['x'])) # numpy.ndarray
        # print(x_qwk['x'][0]) # 0.5
        self.coef_ = x_qwk
        #print(x_qwk['x'])

    def predict(self, X, coef):
        X_p = np.copy(X)
        coef = self.coef_.x # self.coef_.x経由だとなぜかエラーを吐くため
        #print(coef)
        for i, pred in enumerate(X_p):
            if pred < coef[0]:
                X_p[i] = 0
            elif pred >= coef[0] and pred < coef[1]:
                X_p[i] = 1
            elif pred >= coef[1] and pred < coef[2]:
                X_p[i] = 2
            elif pred >= coef[2] and pred < coef[3]:
                X_p[i] = 3
            else:
                X_p[i] = 4
        return X_p

    def coefficients(self):
        return self.coef_.x

In [84]:
def lgb_custom_metric_qwk_regression(preds, data):
    """LightGBM のカスタムメトリックを計算する関数

    回帰問題として解いた予測から QWK を計算する"""
    # 正解ラベル
    y_true = data.get_label()
    # 1:予測ラベル
    #y_pred = np.clip(preds, 0, 7).round()  # 単純に予測値を clip して四捨五入する
    # 2:予測ラベル(最適化)
    optR = OptimizedRounder()
    optR.fit(preds, y_true) # 直接returnしてみる
    optimized = optR.predict(preds, optR.coefficients)
    # QWK を計算する
    return 'qwk', qwk(y_true, optimized), True

def qwk(y_true, y_pred):
    """QWK (Quadratic Weighted Kappa) を計算する関数"""
    return cohen_kappa_score(y_true, y_pred, weights='quadratic')

In [85]:
# 検証用
# 1時間15分
# 評価・予測・提出
class ModelExtractionCallback(object):
    """lightgbm.cv() から学習済みモデルを取り出すためのコールバックに使うクラス

    NOTE: 非公開クラス '_CVBooster' に依存しているため将来的に動かなく恐れがある
    """

    def __init__(self):
        self._model = None

    def __call__(self, env):
        # _CVBooster の参照を保持する
        self._model = env.model

    def _assert_called_cb(self):
        if self._model is None:
            # コールバックが呼ばれていないときは例外にする
            raise RuntimeError('callback has not called yet')

    @property
    def boosters_proxy(self):
        self._assert_called_cb()
        # Booster へのプロキシオブジェクトを返す
        return self._model

    @property
    def raw_boosters(self):
        self._assert_called_cb()
        # Booster のリストを返す
        return self._model.boosters

    @property
    def best_iteration(self):
        self._assert_called_cb()
        # Early stop したときの boosting round を返す
        return self._model.best_iteration

extraction_cb = ModelExtractionCallback()
callbacks = [
    extraction_cb,
]

# LightGBM のデータセット表現にする
lgb_train = lgb.Dataset(x_train, y_train)

# 回帰問題として解く
lgbm_params = {
    'objective': 'regression',
    'metric': 'rmse',
    'first_metric_only': True,
    'verbose': -1, # 学習中のコマンドライン出力
}

# データ分割の方法
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 5-Fold CV で検証する
result = lgb.cv(lgbm_params,
                lgb_train,
                num_boost_round=1000,
                early_stopping_rounds=100,
                folds=folds,
                verbose_eval=10,
                callbacks=callbacks,
                feval=lgb_custom_metric_qwk_regression,
                seed=42,
                )

# early stop したラウンドでの QWK を出力する
print(f'CV Mean QWK: {result["qwk-mean"][-1]}')

[0.5 1.5 2.5 3.5]
[0.5 1.5 2.5 3.5]
[0.5 1.5 2.5 3.5]
[0.5 1.5 2.5 3.5]
[0.5 1.5 2.5 3.5]
[0.5 1.5 2.5 3.5]
[0.5 1.5 2.5 3.5]
[0.5 1.5 2.5 3.5]
[0.5 1.5 2.5 3.5]
[0.5 1.5 2.5 3.5]
[0.5 1.5 2.5 3.5]
[0.5 1.5 2.5 3.5]
[0.5 1.5 2.5 3.5]
[0.5 1.5 2.5 3.5]
[0.5 1.5 2.5 3.5]
[0.51933406 1.55790121 2.13984255 3.57709221]
[0.52034108 1.57179765 2.1234549  3.53602456]
[0.51710602 1.55762336 2.12984442 3.58213818]
[0.52930459 1.5536962  2.10887501 3.60083564]
[0.53318367 1.53911696 2.10787898 3.54725454]
[0.52363941 1.6911026  2.11002571 3.51658275]
[0.52941806 1.73845469 2.0770085  3.48121443]
[0.49178039 2.07329694 2.13405675 3.21066956]
[0.52997597 1.62327251 2.11434419 3.40850207]
[0.52583239 1.59010163 2.12334994 3.5623505 ]
[0.49636313 2.02101942 2.14247563 3.10437701]
[0.51950817 2.06118037 2.12827888 3.38529577]
[0.52269755 1.69633973 2.11608826 3.51502393]
[0.52762834 1.59510374 2.11343119 3.44932994]
[0.5205802  1.73223638 2.14478564 3.46756993]
[0.52190908 1.6757908  2.11100166 3.5920

In [86]:
# コールバックのオブジェクトから学習済みモデルを取り出す

proxy = extraction_cb.boosters_proxy
boosters = extraction_cb.raw_boosters
best_iteration = extraction_cb.best_iteration

# 各モデルの推論結果を Averaging する場合
y_pred_proba_list = proxy.predict(x_dev,
                                  num_iteration=best_iteration)
y_pred_proba_avg = np.array(y_pred_proba_list).mean(axis=0)
y_pred = np.argmax(y_pred_proba_avg, axis=1)

optR = OptimizedRounder()
optR.fit(y_pred, y_dev) # 直接returnしてみる
optimized = optR.predict(y_pred, optR.coefficients)
# QWK を計算する
qwk_dev = qwk(y_dev, optimized)
print('Averaging qwk:', qwk)

# 各モデルで個別に推論する場合
# for i, booster in enumerate(boosters):
#     y_pred_proba = booster.predict(x_dev,
#                                     num_iteration=best_iteration)
#     y_pred = np.argmax(y_pred_proba, axis=1)
#     qwk = accuracy_score(y_test, y_pred)
#     print('Model {0} accuracy: {1}'.format(i, accuracy))


RuntimeError: ignored

CV Mean QWK: 0.32→0.4259343975877961

In [ ]:
# 閾値の最適化
optR = OptimizedRounder()
optR.fit(regression_predictions, original_labels)
print(optR.coefficients)

optimized = optR.predict(regression_predictions, optR.coefficients)
# 最適化した値でqwk出力
optimized_qwk_dev = cohen_kappa_score(y_dev,optimized,weights='quadratic')
print(optimized_qwk_dev)

In [ ]:
# best_qwk = 0
# best_c = 1
# for c in [1, 5,10, 50]:
#     model = LogisticRegression(C=c, random_state=0,max_iter=1500)
#     model.fit(X_train, y_train)
#     y_pred = model.predict(X_dev)
#     qwk = cohen_kappa_score(y_dev,y_pred, weights='quadratic')
#     if qwk > best_qwk:
#         best_qwk = qwk
#         best_c = c
#     print("QWK = %.3f  C = %s" % (qwk, str(c)))
# print("最適なハイパーパラメタは C = %s" % str(best_c))


# model = LogisticRegression(C=best_c, random_state=0,max_iter=1500)
# model.fit(X_train, y_train)
# y_preds = model.predict(X_test)

# y_preds -= 2
# with open('src/' + prepro + '/eval.txt','w') as f:
#   for y_pred in y_preds:
#     y_pred = int(y_pred)
#     y_pred = str(y_pred)
#     f.write(y_pred + '\n')

QWK = 0.330  C = 1
QWK = 0.338  C = 5
QWK = 0.324  C = 10
QWK = 0.299  C = 50
最適なハイパーパラメタは C = 5


-2 origin(nagisa)
QWK = 0.329  C = 1
QWK = 0.339  C = 5
QWK = 0.333  C = 10
QWK = 0.294  C = 50
最適なハイパーパラメタは C = 5

0 origin(顔文字あり)


In [ ]:
print(f'最適なハイパラ C = {str(best_c)} : {prepro}')

最適なハイパラ C = 5 : sudachi/C


## submit

In [ ]:
y_preds = model.predict(X_test)

with open('src/' + prepro + '/eval.txt','w') as f:
  for y_pred in y_preds:
    y_pred = int(y_pred)
    y_pred = str(y_pred)
    f.write(y_pred + '\n')